In [1]:
import pandas as pd
import numpy as np
import sklearn
import pickle
from sklearn import preprocessing
from sklearn import model_selection as ms
from sklearn import linear_model

In [2]:
# Đọc file csv, xóa cột số thứ tự và cột AveMonthSpend
csv = pd.read_csv('CSV.csv').drop(['Unnamed: 0'], axis=1)
#csv = pd.read_csv('CSV.csv').drop(['Unnamed: 0'], axis=1)
csv.head()

,CustomerID,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,Age
0,18988,Bachelors,Management,F,S,0,2,0,5,86931,"45,55"
1,29135,Bachelors,Skilled Manual,M,M,1,2,2,4,100125,"25,45"
2,12156,Graduate Degree,Management,F,M,1,2,0,4,103985,>=55
3,13749,Graduate Degree,Skilled Manual,M,M,1,0,0,4,127161,"25,45"
4,27780,High School,Manual,F,M,1,1,2,2,21876,"25,45"


In [3]:
# Chuyển các dữ liệu dạng biến phân loại về dạng binary dummy, sử dụng one hot code 
def encode_string(cat_feature, name_col):
    enc = preprocessing.LabelEncoder()
    enc = enc.fit(cat_feature)
    pickle.dump(enc, open('encode' + name_col + '.sav', 'wb'))  
    enc_cat_feature = enc.transform(cat_feature)
    
    ohe = preprocessing.OneHotEncoder(categories='auto')
    return ohe.fit_transform(enc_cat_feature.reshape(-1,1)).toarray()

col_to_encodes = ['Occupation','Gender','MaritalStatus',
                  'HomeOwnerFlag','Age']
features = encode_string(csv['Education'], 'Education')
for col in col_to_encodes:
    tmp = encode_string(csv[col], col)
    features = np.concatenate([features, tmp], axis = 1)
print (features.shape)

(500, 20)


In [4]:
# Thêm các giá trị số vào features 
features = np.concatenate([features, np.array(csv[['NumberCarsOwned','NumberChildrenAtHome',
                                                   'TotalChildren','YearlyIncome']])], axis = 1)
print (features.shape)

(500, 24)


In [5]:
# Chuẩn hóa z-score cho các trường
zscore = preprocessing.StandardScaler()
features[:,20:] = zscore.fit_transform(features[:,20:])


In [13]:
# Áp dụng mô hình logistic 
linear_mode = pickle.load(open('final_linearregression.sav', 'rb'))
#linear_mode = pickle.load((open('final_classification.sav', 'rb')))

In [14]:
score_proba = linear_mode.predict(features)


[ 41.8828125 103.0859375  45.2109375  86.6484375  58.546875   43.5390625
  92.578125  123.4921875  99.25       54.1015625  57.7265625  50.0546875
  72.875      44.234375   34.71875    50.484375   83.984375   72.6328125
 106.8984375  57.65625    67.5703125  74.9375    142.9140625  83.6953125
  51.8828125  70.9140625  84.671875  112.9296875  76.4921875  61.515625
  66.0859375  79.8828125  35.921875   71.1328125 102.5859375  98.8046875
 143.9140625  90.0390625  55.1953125  85.125      45.3671875  79.1015625
  79.1484375  47.5625     55.8125     74.84375    59.265625   83.3203125
 114.4453125  78.78125    76.9375     91.5546875  79.6015625  63.3125
  46.9296875  75.375      55.9140625  73.71875    63.6328125  66.8046875
  44.2109375  66.0390625  88.453125   79.4140625  42.8046875  77.7421875
  80.515625  129.28125    62.5859375 103.359375   81.953125   65.9921875
  90.984375   43.7578125  65.671875   79.4921875 109.234375   66.8359375
  58.8125     45.0703125  84.4921875  72.4921875  46.37

Mean Square Error      = 38.89508424377441
Root Mean Square Error = 6.236592358313506
Mean Absolute Error    = 4.739197265625
Median Absolute Error  = 3.7109375
R^2                    = 0.9477613852651927
Adjusted R^2           = 0.9477483190784156


In [15]:
print(score_proba)

[ 41.8828125 103.0859375  45.2109375  86.6484375  58.546875   43.5390625
  92.578125  123.4921875  99.25       54.1015625  57.7265625  50.0546875
  72.875      44.234375   34.71875    50.484375   83.984375   72.6328125
 106.8984375  57.65625    67.5703125  74.9375    142.9140625  83.6953125
  51.8828125  70.9140625  84.671875  112.9296875  76.4921875  61.515625
  66.0859375  79.8828125  35.921875   71.1328125 102.5859375  98.8046875
 143.9140625  90.0390625  55.1953125  85.125      45.3671875  79.1015625
  79.1484375  47.5625     55.8125     74.84375    59.265625   83.3203125
 114.4453125  78.78125    76.9375     91.5546875  79.6015625  63.3125
  46.9296875  75.375      55.9140625  73.71875    63.6328125  66.8046875
  44.2109375  66.0390625  88.453125   79.4140625  42.8046875  77.7421875
  80.515625  129.28125    62.5859375 103.359375   81.953125   65.9921875
  90.984375   43.7578125  65.671875   79.4921875 109.234375   66.8359375
  58.8125     45.0703125  84.4921875  72.4921875  46.37

In [16]:
point = pd.DataFrame(score_proba)
print (point.shape)
point.to_csv('KQ.csv')

(500, 1)
